# word2vec를 활용한 모델 구현

## 전처리된 텍스트 데이터 불러오기 및 각 단어의 리스트 나누기

In [2]:
import pandas as pd

DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

## 하이퍼파라미터 설정

In [3]:
num_features = 300 # 워드 벡터 특징값 수
min_word_count = 40 # 단어에 대한 최소 빈도 수
num_workers = 4 # 프로세스 개수
context = 10 # 컨텍스트 윈도 크기
downsampling = 1e-3 # 다운 샘플링 비율

## gensim 라이브러리 설치

In [4]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 진행 상황 확인을 위한 logging

In [6]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 학습

In [7]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

## 모델 저장

In [8]:
model_name = "300features_40minwords_10context"
model.save(model_name)

## 하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수

In [9]:
import numpy as np

def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

## 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수

In [10]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

## 실제 학습에 사용될 입력값

In [11]:
test_data_vecs = get_dataset(sentences, model, num_features)

<ipython-input-9-52c0d11dc7c6>:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


## 학습과 검증 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

## 모델 선언 및 학습

In [15]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced', max_iter=500)
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

## 검증 데이터셋을 이용한 성능 평가

In [16]:
print("Accuracy: %f" % lgs.score(X_test, y_test)) 

Accuracy: 0.866400


## 전처리한 평가 데이터를 불러온 후 리뷰 값을 리스트로 저장

In [17]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

## 평가 데이터를 각 단어의 리스트로 저장

In [18]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

## 각 단어를 벡터로 만들어 각 리뷰에 대한 특징값 저장

In [19]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

<ipython-input-9-52c0d11dc7c6>:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


## CSV 파일로 만들어 저장

In [ ]:
import os

DATA_OUT_PATH = './data_out/'

test_predicted = lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)